# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [58]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [59]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [144]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """

    source_sentences = source_text.split('\n')
    target_sentences = [sentence + ' <EOS>' for sentence in target_text.split('\n')]
 
    print(source_sentences)
    print(target_sentences)
    source_id_text = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in source_sentences]
    target_id_text = [[target_vocab_to_int[word] for word in sentence.split()] for sentence in target_sentences]
    
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

['new jersey is sometimes quiet during autumn , and it is snowy in april .', 'the united states is usually chilly during july , and it is usually freezing in november .', 'california is usually quiet during march , and it is usually hot in june .', 'the united states is sometimes mild during june , and it is cold in september .']
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril . <EOS>", 'les états-unis est généralement froid en juillet , et il gèle habituellement en novembre . <EOS>', 'california est généralement calme en mars , et il est généralement chaud en juin . <EOS>', 'les états-unis est parfois légère en juin , et il fait froid en septembre . <EOS>']
Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [61]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [62]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [145]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


C:\Users\ezk922\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  from ipykernel import kernelapp as app


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [146]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """

    input = tf.placeholder(tf.int32,shape=[None,None],name='input')
    targets = tf.placeholder(tf.int32,shape=[None,None],name='targets')
    learning_rate = tf.placeholder(tf.float32,shape=[],name='learning_rate')
    keep_probability = tf.placeholder(tf.float32,shape=[],name='keep_prob')
    target_sequence_length = tf.placeholder(tf.int32,[None,],name='target_sequence_length')
    max_target_length = tf.reduce_max(target_sequence_length)
    source_sequence_length = tf.placeholder(tf.int32,[None,],name='source_sequence_length')
    
    return input, targets, learning_rate, keep_probability, target_sequence_length, max_target_length, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [147]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """

    last_word = tf.strided_slice(target_data, [0,0], [batch_size, -1], [1,1])
    modified_data = tf.fill([batch_size, 1], target_vocab_to_int['<GO>'])
    preprocessed_data = tf.concat([modified_data, last_word], 1)
    return preprocessed_data

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [148]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    def build_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return lstm_drop
    
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    embed_encoder = tf.contrib.layers.embed_sequence(rnn_inputs, vocab_size = source_vocab_size, embed_dim = encoding_embedding_size)
    output, state = tf.nn.dynamic_rnn(stacked_lstm, embed_encoder, source_sequence_length, dtype=tf.float32)

    return output, state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [149]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """

    training_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    
    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, training_helper, encoder_state, output_layer)
    
    decoded = tf.contrib.seq2seq.dynamic_decode(training_decoder, maximum_iterations=max_summary_length)[0]
    return decoded



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [150]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    start_ids = tf.tile([start_of_sequence_id], [batch_size])
 
    embedding_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_ids, end_of_sequence_id)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, embedding_helper, encoder_state, output_layer)
    output = tf.contrib.seq2seq.dynamic_decode(basic_decoder,maximum_iterations=max_target_sequence_length)[0]
    return output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [151]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
 
    def build_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        return tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
 
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    dense_layer = Dense(target_vocab_size, kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    with tf.variable_scope("decode") as scope:
        tr_decoder_output = decoding_layer_train(encoder_state, stacked_lstm, dec_embed_input, target_sequence_length, max_target_sequence_length, dense_layer, keep_prob)
        scope.reuse_variables()
        inf_decoder_output = decoding_layer_infer(encoder_state, stacked_lstm, dec_embeddings, target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'], 
                                                  max_target_sequence_length, target_vocab_size, dense_layer, batch_size, keep_prob)
    
    return tr_decoder_output, inf_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [152]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    output, state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, source_sequence_length, source_vocab_size, enc_embedding_size)
    processed_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    tr_decoder_output, inf_decoder_output = decoding_layer(processed_input, state, target_sequence_length, max_target_sentence_length, 
                                                           rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)
    
    return tr_decoder_output, inf_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [162]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 256
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = .001
# Dropout Keep Probability
keep_probability = .8
display_step = True



### Build the Graph
Build the graph using the neural network you implemented.

In [163]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])


train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [164]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [165]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/538 - Train Accuracy: 0.2314, Validation Accuracy: 0.3157, Loss: 5.3366
Epoch   0 Batch    2/538 - Train Accuracy: 0.2516, Validation Accuracy: 0.3157, Loss: 4.7196
Epoch   0 Batch    3/538 - Train Accuracy: 0.2289, Validation Accuracy: 0.3157, Loss: 4.6268
Epoch   0 Batch    4/538 - Train Accuracy: 0.2479, Validation Accuracy: 0.3281, Loss: 4.4721
Epoch   0 Batch    5/538 - Train Accuracy: 0.2969, Validation Accuracy: 0.3457, Loss: 4.1312
Epoch   0 Batch    6/538 - Train Accuracy: 0.3047, Validation Accuracy: 0.3469, Loss: 3.9900
Epoch   0 Batch    7/538 - Train Accuracy: 0.2820, Validation Accuracy: 0.3471, Loss: 3.9754
Epoch   0 Batch    8/538 - Train Accuracy: 0.2799, Validation Accuracy: 0.3471, Loss: 3.8585
Epoch   0 Batch    9/538 - Train Accuracy: 0.3039, Validation Accuracy: 0.3665, Loss: 3.7018
Epoch   0 Batch   10/538 - Train Accuracy: 0.2951, Validation Accuracy: 0.3750, Loss: 3.7129
Epoch   0 Batch   11/538 - Train Accuracy: 0.3094, Validation Accuracy

Epoch   0 Batch   90/538 - Train Accuracy: 0.4609, Validation Accuracy: 0.4920, Loss: 1.9221
Epoch   0 Batch   91/538 - Train Accuracy: 0.4715, Validation Accuracy: 0.5252, Loss: 1.9997
Epoch   0 Batch   92/538 - Train Accuracy: 0.4697, Validation Accuracy: 0.5167, Loss: 1.9384
Epoch   0 Batch   93/538 - Train Accuracy: 0.4117, Validation Accuracy: 0.4831, Loss: 1.9529
Epoch   0 Batch   94/538 - Train Accuracy: 0.4611, Validation Accuracy: 0.5076, Loss: 1.9873
Epoch   0 Batch   95/538 - Train Accuracy: 0.5167, Validation Accuracy: 0.5186, Loss: 1.7480
Epoch   0 Batch   96/538 - Train Accuracy: 0.4967, Validation Accuracy: 0.5123, Loss: 1.7828
Epoch   0 Batch   97/538 - Train Accuracy: 0.4340, Validation Accuracy: 0.5020, Loss: 1.8671
Epoch   0 Batch   98/538 - Train Accuracy: 0.4864, Validation Accuracy: 0.5080, Loss: 1.7396
Epoch   0 Batch   99/538 - Train Accuracy: 0.4617, Validation Accuracy: 0.5174, Loss: 1.8658
Epoch   0 Batch  100/538 - Train Accuracy: 0.4623, Validation Accuracy

Epoch   0 Batch  179/538 - Train Accuracy: 0.4908, Validation Accuracy: 0.5238, Loss: 1.2646
Epoch   0 Batch  180/538 - Train Accuracy: 0.5177, Validation Accuracy: 0.5165, Loss: 1.2393
Epoch   0 Batch  181/538 - Train Accuracy: 0.4768, Validation Accuracy: 0.5382, Loss: 1.3000
Epoch   0 Batch  182/538 - Train Accuracy: 0.4711, Validation Accuracy: 0.5247, Loss: 1.2747
Epoch   0 Batch  183/538 - Train Accuracy: 0.5048, Validation Accuracy: 0.5188, Loss: 1.1925
Epoch   0 Batch  184/538 - Train Accuracy: 0.5236, Validation Accuracy: 0.5355, Loss: 1.1894
Epoch   0 Batch  185/538 - Train Accuracy: 0.5000, Validation Accuracy: 0.5245, Loss: 1.2402
Epoch   0 Batch  186/538 - Train Accuracy: 0.5221, Validation Accuracy: 0.5336, Loss: 1.2345
Epoch   0 Batch  187/538 - Train Accuracy: 0.5554, Validation Accuracy: 0.5439, Loss: 1.1635
Epoch   0 Batch  188/538 - Train Accuracy: 0.4662, Validation Accuracy: 0.5218, Loss: 1.2497
Epoch   0 Batch  189/538 - Train Accuracy: 0.5037, Validation Accuracy

Epoch   0 Batch  268/538 - Train Accuracy: 0.5439, Validation Accuracy: 0.5607, Loss: 0.8620
Epoch   0 Batch  269/538 - Train Accuracy: 0.5068, Validation Accuracy: 0.5527, Loss: 0.9321
Epoch   0 Batch  270/538 - Train Accuracy: 0.4984, Validation Accuracy: 0.5460, Loss: 0.9240
Epoch   0 Batch  271/538 - Train Accuracy: 0.5236, Validation Accuracy: 0.5543, Loss: 0.9281
Epoch   0 Batch  272/538 - Train Accuracy: 0.5098, Validation Accuracy: 0.5595, Loss: 0.9699
Epoch   0 Batch  273/538 - Train Accuracy: 0.5322, Validation Accuracy: 0.5613, Loss: 0.9271
Epoch   0 Batch  274/538 - Train Accuracy: 0.4895, Validation Accuracy: 0.5575, Loss: 0.9499
Epoch   0 Batch  275/538 - Train Accuracy: 0.5166, Validation Accuracy: 0.5508, Loss: 0.9296
Epoch   0 Batch  276/538 - Train Accuracy: 0.5350, Validation Accuracy: 0.5536, Loss: 0.9182
Epoch   0 Batch  277/538 - Train Accuracy: 0.5350, Validation Accuracy: 0.5577, Loss: 0.9049
Epoch   0 Batch  278/538 - Train Accuracy: 0.5291, Validation Accuracy

Epoch   0 Batch  357/538 - Train Accuracy: 0.5826, Validation Accuracy: 0.5826, Loss: 0.7516
Epoch   0 Batch  358/538 - Train Accuracy: 0.5477, Validation Accuracy: 0.5708, Loss: 0.7596
Epoch   0 Batch  359/538 - Train Accuracy: 0.5858, Validation Accuracy: 0.5650, Loss: 0.7447
Epoch   0 Batch  360/538 - Train Accuracy: 0.5699, Validation Accuracy: 0.5751, Loss: 0.7685
Epoch   0 Batch  361/538 - Train Accuracy: 0.5768, Validation Accuracy: 0.5826, Loss: 0.7272
Epoch   0 Batch  362/538 - Train Accuracy: 0.5705, Validation Accuracy: 0.5794, Loss: 0.7191
Epoch   0 Batch  363/538 - Train Accuracy: 0.5688, Validation Accuracy: 0.5835, Loss: 0.7273
Epoch   0 Batch  364/538 - Train Accuracy: 0.5430, Validation Accuracy: 0.5843, Loss: 0.7841
Epoch   0 Batch  365/538 - Train Accuracy: 0.5789, Validation Accuracy: 0.5863, Loss: 0.7419
Epoch   0 Batch  366/538 - Train Accuracy: 0.5623, Validation Accuracy: 0.5751, Loss: 0.7581
Epoch   0 Batch  367/538 - Train Accuracy: 0.5893, Validation Accuracy

...

Epoch   9 Batch  161/538 - Train Accuracy: 0.9641, Validation Accuracy: 0.9553, Loss: 0.0164
Epoch   9 Batch  162/538 - Train Accuracy: 0.9801, Validation Accuracy: 0.9581, Loss: 0.0164
Epoch   9 Batch  163/538 - Train Accuracy: 0.9680, Validation Accuracy: 0.9629, Loss: 0.0233
Epoch   9 Batch  164/538 - Train Accuracy: 0.9713, Validation Accuracy: 0.9691, Loss: 0.0183
Epoch   9 Batch  165/538 - Train Accuracy: 0.9805, Validation Accuracy: 0.9732, Loss: 0.0162
Epoch   9 Batch  166/538 - Train Accuracy: 0.9867, Validation Accuracy: 0.9732, Loss: 0.0145
Epoch   9 Batch  167/538 - Train Accuracy: 0.9615, Validation Accuracy: 0.9755, Loss: 0.0280
Epoch   9 Batch  168/538 - Train Accuracy: 0.9555, Validation Accuracy: 0.9716, Loss: 0.0241
Epoch   9 Batch  169/538 - Train Accuracy: 0.9932, Validation Accuracy: 0.9716, Loss: 0.0115
Epoch   9 Batch  170/538 - Train Accuracy: 0.9622, Validation Accuracy: 0.9753, Loss: 0.0178
Epoch   9 Batch  171/538 - Train Accuracy: 0.9584, Validation Accuracy

Epoch   9 Batch  250/538 - Train Accuracy: 0.9869, Validation Accuracy: 0.9599, Loss: 0.0193
Epoch   9 Batch  251/538 - Train Accuracy: 0.9758, Validation Accuracy: 0.9599, Loss: 0.0153
Epoch   9 Batch  252/538 - Train Accuracy: 0.9682, Validation Accuracy: 0.9688, Loss: 0.0184
Epoch   9 Batch  253/538 - Train Accuracy: 0.9706, Validation Accuracy: 0.9714, Loss: 0.0164
Epoch   9 Batch  254/538 - Train Accuracy: 0.9676, Validation Accuracy: 0.9737, Loss: 0.0183
Epoch   9 Batch  255/538 - Train Accuracy: 0.9867, Validation Accuracy: 0.9746, Loss: 0.0151
Epoch   9 Batch  256/538 - Train Accuracy: 0.9715, Validation Accuracy: 0.9723, Loss: 0.0172
Epoch   9 Batch  257/538 - Train Accuracy: 0.9831, Validation Accuracy: 0.9719, Loss: 0.0148
Epoch   9 Batch  258/538 - Train Accuracy: 0.9645, Validation Accuracy: 0.9700, Loss: 0.0187
Epoch   9 Batch  259/538 - Train Accuracy: 0.9805, Validation Accuracy: 0.9718, Loss: 0.0125
Epoch   9 Batch  260/538 - Train Accuracy: 0.9561, Validation Accuracy

Epoch   9 Batch  339/538 - Train Accuracy: 0.9710, Validation Accuracy: 0.9641, Loss: 0.0184
Epoch   9 Batch  340/538 - Train Accuracy: 0.9709, Validation Accuracy: 0.9636, Loss: 0.0204
Epoch   9 Batch  341/538 - Train Accuracy: 0.9758, Validation Accuracy: 0.9611, Loss: 0.0192
Epoch   9 Batch  342/538 - Train Accuracy: 0.9734, Validation Accuracy: 0.9627, Loss: 0.0179
Epoch   9 Batch  343/538 - Train Accuracy: 0.9869, Validation Accuracy: 0.9664, Loss: 0.0160
Epoch   9 Batch  344/538 - Train Accuracy: 0.9631, Validation Accuracy: 0.9700, Loss: 0.0176
Epoch   9 Batch  345/538 - Train Accuracy: 0.9847, Validation Accuracy: 0.9725, Loss: 0.0156
Epoch   9 Batch  346/538 - Train Accuracy: 0.9647, Validation Accuracy: 0.9695, Loss: 0.0240
Epoch   9 Batch  347/538 - Train Accuracy: 0.9844, Validation Accuracy: 0.9682, Loss: 0.0167
Epoch   9 Batch  348/538 - Train Accuracy: 0.9591, Validation Accuracy: 0.9688, Loss: 0.0193
Epoch   9 Batch  349/538 - Train Accuracy: 0.9895, Validation Accuracy

Epoch   9 Batch  428/538 - Train Accuracy: 0.9786, Validation Accuracy: 0.9730, Loss: 0.0113
Epoch   9 Batch  429/538 - Train Accuracy: 0.9794, Validation Accuracy: 0.9780, Loss: 0.0217
Epoch   9 Batch  430/538 - Train Accuracy: 0.9730, Validation Accuracy: 0.9817, Loss: 0.0214
Epoch   9 Batch  431/538 - Train Accuracy: 0.9736, Validation Accuracy: 0.9815, Loss: 0.0170
Epoch   9 Batch  432/538 - Train Accuracy: 0.9659, Validation Accuracy: 0.9830, Loss: 0.0199
Epoch   9 Batch  433/538 - Train Accuracy: 0.9654, Validation Accuracy: 0.9760, Loss: 0.0350
Epoch   9 Batch  434/538 - Train Accuracy: 0.9746, Validation Accuracy: 0.9712, Loss: 0.0165
Epoch   9 Batch  435/538 - Train Accuracy: 0.9752, Validation Accuracy: 0.9553, Loss: 0.0178
Epoch   9 Batch  436/538 - Train Accuracy: 0.9467, Validation Accuracy: 0.9588, Loss: 0.0224
Epoch   9 Batch  437/538 - Train Accuracy: 0.9857, Validation Accuracy: 0.9691, Loss: 0.0173
Epoch   9 Batch  438/538 - Train Accuracy: 0.9797, Validation Accuracy

Epoch   9 Batch  517/538 - Train Accuracy: 0.9812, Validation Accuracy: 0.9670, Loss: 0.0146
Epoch   9 Batch  518/538 - Train Accuracy: 0.9756, Validation Accuracy: 0.9693, Loss: 0.0193
Epoch   9 Batch  519/538 - Train Accuracy: 0.9898, Validation Accuracy: 0.9759, Loss: 0.0129
Epoch   9 Batch  520/538 - Train Accuracy: 0.9730, Validation Accuracy: 0.9703, Loss: 0.0177
Epoch   9 Batch  521/538 - Train Accuracy: 0.9785, Validation Accuracy: 0.9656, Loss: 0.0190
Epoch   9 Batch  522/538 - Train Accuracy: 0.9670, Validation Accuracy: 0.9682, Loss: 0.0177
Epoch   9 Batch  523/538 - Train Accuracy: 0.9875, Validation Accuracy: 0.9703, Loss: 0.0205
Epoch   9 Batch  524/538 - Train Accuracy: 0.9775, Validation Accuracy: 0.9744, Loss: 0.0158
Epoch   9 Batch  525/538 - Train Accuracy: 0.9730, Validation Accuracy: 0.9744, Loss: 0.0217
Epoch   9 Batch  526/538 - Train Accuracy: 0.9682, Validation Accuracy: 0.9769, Loss: 0.0155
Epoch   9 Batch  527/538 - Train Accuracy: 0.9826, Validation Accuracy

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [166]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [167]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [168]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    return [vocab_to_int.get(word, vocab_to_int.get('<UNK>')) for word in sentence.lower().split()]



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [169]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [220, 89, 216, 65, 187, 193, 31]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [136, 179, 10, 336, 227, 72, 189, 1]
  French Words: il a vu un vieux camion . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.